# Uncertainity Quantification In Forecasting Problem Using Neural Networks Techniques
- BNN (Bayesian neural network)
- MC Dropot (Monte Carlo Dropot)  
- Qunatile Regression

# Main setup

In [88]:
# General imports
import numpy as np
import pandas as pd
#warnings.filterwarnings('ignore')

In [89]:
# ## Merging by concat to not lose dtypes
# def merge_by_concat(df1, df2, merge_on):
#     merged_gf = df1[merge_on]
#     merged_gf = merged_gf.merge(df2, on=merge_on, how='left')
#     new_columns = [col for col in list(merged_gf) if col not in merge_on]
#     df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
#     return df1
import os

#variables:
mount = True #whether you want to mount from drive
subsample = True

In [90]:
os.getcwd()

'c:\\Users\\derri\\VSCode\\CS578-Uncertainty-Project'

In [91]:
# Loading Data sets
# vishal's original path
path = "/Users/vishalgajavelly/CS_578_Project/"
# requires that you have the dataset in a "CS 578 Project" folder
if mount:
  path = os.getcwd() + "/data/"
product_sales_df = pd.read_csv(path + "sales_train_evaluation.csv")
prices_df = pd.read_csv(path + "sell_prices.csv")
calendar_df = pd.read_csv(path + "calendar.csv")

print(product_sales_df.columns)
print(prices_df.columns)
print(calendar_df.columns)

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd_1',
       'd_2', 'd_3', 'd_4',
       ...
       'd_1932', 'd_1933', 'd_1934', 'd_1935', 'd_1936', 'd_1937', 'd_1938',
       'd_1939', 'd_1940', 'd_1941'],
      dtype='object', length=1947)
Index(['store_id', 'item_id', 'wm_yr_wk', 'sell_price'], dtype='object')
Index(['date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year', 'd',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI'],
      dtype='object')


In [92]:
product_sales_df['state_id'].nunique()

3

In [93]:
if subsample:
  #product_sales_df = product_sales_df.where((product_sales_df['state_id'] == 'CA') and (product_sales_df['cat_id'] == 'HOBBIES')).dropna()
  product_sales_df = product_sales_df[product_sales_df['state_id'] == 'CA']
  product_sales_df = product_sales_df[product_sales_df['cat_id'] == 'HOBBIES']
product_sales_df['state_id'].nunique()

1

In [94]:
product_sales_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


In [95]:
prices_df.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [96]:
product_sales_df['item_id'].nunique()

565

In [97]:
prices_df.dtypes

store_id       object
item_id        object
wm_yr_wk        int64
sell_price    float64
dtype: object

In [98]:
if subsample:
    calendar_df2 = calendar_df.drop(calendar_df[calendar_df.year < 2014].index)
    calendar_df2 = calendar_df2.drop(calendar_df2[calendar_df2.year > 2016].index)

    calendar_df = calendar_df.drop(calendar_df[calendar_df.year < 2014].index)
    calendar_df = calendar_df.drop(calendar_df[calendar_df.year > 2016].index)
calendar_df.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
1068,2014-01-01,11349,Wednesday,5,1,2014,d_1069,NewYear,National,NaN,NaN,1,1,0
1069,2014-01-02,11349,Thursday,6,1,2014,d_1070,NaN,NaN,NaN,NaN,1,0,1
1070,2014-01-03,11349,Friday,7,1,2014,d_1071,NaN,NaN,NaN,NaN,1,1,1
1071,2014-01-04,11350,Saturday,1,1,2014,d_1072,NaN,NaN,NaN,NaN,1,0,0
1072,2014-01-05,11350,Sunday,2,1,2014,d_1073,NaN,NaN,NaN,NaN,1,1,1


In [99]:
if subsample:
    min = int(calendar_df2["d"].min()[2:])
    max = int(calendar_df2["d"].max()[2:])
    remove = []
    for i in range(1, min-30, 1):
        remove.append("d_"+str(i))
    product_sales_df.drop(remove, axis = 1, inplace=True)
    
    for i in range(max + 1, 10000, 1):
        try:
            product_sales_df.drop("d_"+str(i), axis = 1, inplace=True)
        except:
            break
product_sales_df.head()


,id,item_id,dept_id,cat_id,store_id,state_id,d_1039,d_1040,d_1041,d_1042,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1,0,1,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,3,0,1,6,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1,0,1,1,...,0,0,0,2,1,0,0,2,1,0


In [100]:
calendar_df.nunique()

date            901
wm_yr_wk        130
weekday           7
wday              7
month            12
year              3
d               901
event_name_1     30
event_type_1      4
event_name_2      2
event_type_2      2
snap_CA           2
snap_TX           2
snap_WI           2
dtype: int64

In [101]:
calendar_df.dtypes

date            object
wm_yr_wk         int64
weekday         object
wday             int64
month            int64
year             int64
d               object
event_name_1    object
event_type_1    object
event_name_2    object
event_type_2    object
snap_CA          int64
snap_TX          int64
snap_WI          int64
dtype: object

In [102]:
calendar_df['event_name_2'].unique()

array([nan, 'OrthodoxEaster', "Father's day"], dtype=object)

# Merging
- Melt the product_sales_df (d_1,d_2,...d_1941)
columns to single column d

In [103]:
index_columns = ['id','item_id','dept_id','cat_id','store_id','state_id']
df1 = pd.melt(product_sales_df,
                  id_vars = index_columns,
                  var_name = 'd',
                  value_name = 'sales')

In [104]:
df1.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1039,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1039,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1039,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1039,3
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1039,1


In [105]:
print(df1.count())

id          2040780
item_id     2040780
dept_id     2040780
cat_id      2040780
store_id    2040780
state_id    2040780
d           2040780
sales       2040780
dtype: int64


- Left join df1 with calender_df on d
- If subsampling, we do an inner join to discard non-qualified datapoints

In [106]:
merge_on = ['d']
merged_gf = df1[merge_on]
merged_gf = merged_gf.merge(calendar_df, on = merge_on, how='left')
new_columns = [col for col in list(merged_gf) if col not in merge_on]
df2 = pd.concat([df1, merged_gf[new_columns]], axis=1)

In [107]:
df2.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1039,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1039,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1039,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1039,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1039,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
calendar_df.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
1068,2014-01-01,11349,Wednesday,5,1,2014,d_1069,NewYear,National,NaN,NaN,1,1,0
1069,2014-01-02,11349,Thursday,6,1,2014,d_1070,NaN,NaN,NaN,NaN,1,0,1
1070,2014-01-03,11349,Friday,7,1,2014,d_1071,NaN,NaN,NaN,NaN,1,1,1
1071,2014-01-04,11350,Saturday,1,1,2014,d_1072,NaN,NaN,NaN,NaN,1,0,0
1072,2014-01-05,11350,Sunday,2,1,2014,d_1073,NaN,NaN,NaN,NaN,1,1,1


In [109]:
product_sales_df.size

2054340

- Left join df2 with prices_df on item_id, store_id, wm_yr_wk

In [110]:
merge_on = ['item_id', 'store_id', 'wm_yr_wk']
merged_gf = df2[merge_on]
merged_gf = merged_gf.merge(prices_df, on = merge_on, how='left')
new_columns = [col for col in list(merged_gf) if col not in merge_on]
df3 = pd.concat([df2, merged_gf[new_columns]], axis=1)

In [111]:
df3.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1039,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1039,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1039,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1039,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1039,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [112]:
df3['sell_price'].unique()

array([  nan,  8.26,  3.97,  4.64,  3.08,  1.  ,  7.88,  0.46,  1.77,
        2.97,  3.48,  6.52,  1.97,  0.98,  0.7 ,  1.98, 11.88, 12.68,
        7.18,  3.44,  8.98,  3.98,  7.98, 10.98,  0.58, 15.97,  4.66,
        2.12,  1.12, 12.97,  3.22,  5.88,  4.22,  7.48,  1.48,  8.28,
        5.98,  0.94,  4.88,  8.67, 30.98,  2.83,  0.97,  5.47,  1.25,
        7.  , 13.68,  3.67,  3.43,  9.98, 12.78, 14.72,  3.78, 17.48,
        4.38, 12.88,  2.57,  7.97,  2.38, 20.38,  7.68,  4.24, 15.48,
        4.98,  6.68, 12.98,  6.12,  3.12,  2.72,  1.47,  5.97,  6.47,
       11.12,  8.77,  2.73,  4.58,  3.87,  9.48,  2.63,  1.28,  9.97,
        1.56,  5.5 ,  6.18,  2.88, 18.98, 19.98,  3.24,  9.42,  8.97,
        4.97, 11.98, 17.97,  2.98, 10.97,  5.94,  6.87,  3.54, 12.84,
        8.38,  3.42, 13.48,  2.78,  7.47,  9.88,  3.47, 10.58,  1.74,
        4.48,  0.3 , 13.88, 15.83,  1.88, 16.76, 11.78,  4.47,  5.77,
        8.44, 10.88,  5.48,  5.36,  3.68,  3.88,  7.38,  1.27,  6.24,
        2.87, 13.98,

In [113]:
df3.count()

id              2040780
item_id         2040780
dept_id         2040780
cat_id          2040780
store_id        2040780
state_id        2040780
d               2040780
sales           2040780
date            1972980
wm_yr_wk        1972980
weekday         1972980
wday            1972980
month           1972980
year            1972980
event_name_1     164980
event_type_1     164980
event_name_2       4520
event_type_2       4520
snap_CA         1972980
snap_TX         1972980
snap_WI         1972980
sell_price      1925498
dtype: int64

In [114]:
print(df3.columns)
print(df3.dtypes)

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'sell_price'],
      dtype='object')
id               object
item_id          object
dept_id          object
cat_id           object
store_id         object
state_id         object
d                object
sales             int64
date             object
wm_yr_wk        float64
weekday          object
wday            float64
month           float64
year            float64
event_name_1     object
event_type_1     object
event_name_2     object
event_type_2     object
snap_CA         float64
snap_TX         float64
snap_WI         float64
sell_price      float64
dtype: object


In [ ]:
if mount:
  df3.to_csv(path+"merged_df.csv")
else:
  df3.to_csv("/Users/vishalgajavelly/CS_578_Project/merged_df.csv")

: 

# Encoding

# Feature Engineering